# intro al analisis

In [1]:
import sqlite3
import pandas as pd
import sqlalchemy

engine=sqlalchemy.create_engine(r'sqlite:///C:\Users\56945\Downloads\database.sqlite')

### 1) Toda la información para cada partido

Con la siguiente query, se puede obtener toda la información disponible para cada partido

In [2]:
matches=pd.read_sql(
    """ 
    SELECT  a.*, b.name as country_name, c.name as league_name, d_h.team_long_name as home_team_name, d_a.team_long_name as away_team_name
    FROM Match as a
    LEFT JOIN Country as b on a.country_id=b.id
    LEFT JOIN League as c on a.league_id=c.id
    LEFT JOIN Team as d_h on a.home_team_api_id=d_h.team_api_id 
    LEFT JOIN Team as d_a on a.away_team_api_id=d_a.team_api_id 
    """
    , engine
)

In [3]:
matches.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,GBH,GBD,GBA,BSH,BSD,BSA,country_name,league_name,home_team_name,away_team_name
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,1.78,3.25,4.00,1.73,3.40,4.20,Belgium,Belgium Jupiler League,KRC Genk,Beerschot AC
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,1.85,3.25,3.75,1.91,3.25,3.60,Belgium,Belgium Jupiler League,SV Zulte-Waregem,Sporting Lokeren
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,2.50,3.20,2.50,2.30,3.20,2.75,Belgium,Belgium Jupiler League,KSV Cercle Brugge,RSC Anderlecht
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,1.50,3.75,5.50,1.44,3.75,6.50,Belgium,Belgium Jupiler League,KAA Gent,RAEC Mons
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,4.50,3.50,1.65,4.75,3.30,1.67,Belgium,Belgium Jupiler League,FCV Dender EH,Standard de Liège


Se observa que el dataframe llamado "matches", está compuesto por 26979 partidos unicos y 119 filas distintas

### 2) Insights la información detallada de cada partido
Para poder realizar la busqueda de insights, se procede a limpiar el dataframe matches, esto con el objetivo de disminuir las 119 filas y dejar las que pueden ser más utiles para el análisis

In [4]:
matches[['id','country_name','league_name','season','date','home_team_name','home_team_goal','away_team_name','away_team_goal']]
## numero de partidos con victoria, empate, y perdida. General y para home and away
# total de goles por temporada, equipo con mas goles por temporada
# equipos que juegan mejor de home o away / crear un ratio de 0 -> solo gana away 1-> solo gana home
#liga con mas goles
# como se distribuyen los partidos en cada año, numero de partidos en un año
# ¿existen meses donde existan favores para home o away?


,id,country_name,league_name,season,date,home_team_name,home_team_goal,away_team_name,away_team_goal
0,1,Belgium,Belgium Jupiler League,2008/2009,2008-08-17 00:00:00,KRC Genk,1,Beerschot AC,1
1,2,Belgium,Belgium Jupiler League,2008/2009,2008-08-16 00:00:00,SV Zulte-Waregem,0,Sporting Lokeren,0
2,3,Belgium,Belgium Jupiler League,2008/2009,2008-08-16 00:00:00,KSV Cercle Brugge,0,RSC Anderlecht,3
3,4,Belgium,Belgium Jupiler League,2008/2009,2008-08-17 00:00:00,KAA Gent,5,RAEC Mons,0
4,5,Belgium,Belgium Jupiler League,2008/2009,2008-08-16 00:00:00,FCV Dender EH,1,Standard de Liège,3
...,...,...,...,...,...,...,...,...,...
25974,25975,Switzerland,Switzerland Super League,2015/2016,2015-09-22 00:00:00,FC St. Gallen,1,FC Thun,0
25975,25976,Switzerland,Switzerland Super League,2015/2016,2015-09-23 00:00:00,FC Vaduz,1,FC Luzern,2
25976,25977,Switzerland,Switzerland Super League,2015/2016,2015-09-23 00:00:00,Grasshopper Club Zürich,2,FC Sion,0
25977,25978,Switzerland,Switzerland Super League,2015/2016,2015-09-22 00:00:00,Lugano,0,FC Zürich,0


### 3) Información detallada por jugador (uno por fila)

A continuación se entrega la mejor versión de cada jugador junto con sus respectivos atributos.


In [5]:
players=pd.read_sql(
    """ 
    SELECT a.*, b.player_name, b.height
    FROM(
        SELECT *, row_number()OVER(PARTITION BY player_api_id ORDER BY overall_rating DESC) as ranked
        from Player_Attributes) as a
    LEFT JOIN Player as b on b.player_api_id=a.player_api_id
    WHERE ranked=1
    """
    , engine
)

In [6]:
players.head()

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,ranked,player_name,height
0,139856,148544,2625,2007-08-30 00:00:00,63,64,right,high,medium,48,...,66,63.0,14,24,67,24,24,1,"Patryk Rachwal,18",175.26
1,44145,135819,2752,2015-10-16 00:00:00,72,72,right,medium,medium,39,...,74,67.0,12,7,8,10,16,1,Diego Mainz,187.96
2,88286,110019,2768,2016-03-03 00:00:00,75,75,left,medium,medium,44,...,77,79.0,12,15,13,14,10,1,Jose Dorado,180.34
3,72150,182861,2770,2011-02-22 00:00:00,74,81,left,medium,low,63,...,48,25.0,12,13,6,14,15,1,Ignacio Gonzalez,180.34
4,5123,110809,2790,2009-08-30 00:00:00,73,77,left,None,7,78,...,78,69.0,8,21,73,21,21,1,Alberto Rey,175.26


### 4) Insights de la información por jugador

In [7]:
# hacer grafico de violin(+scatter) para ver en cuantos equipos se mueven los jugadores
# numero de partidos y equipos jugados

### 5) Mejor equipo de la historia

 Para buscar al mejor equipo posible se realizó una investigación para obtener los atributos mas importantes para cada puesto en el campo (defensa, mediocampo y delanteros) y posteriormente se procede a realizar un promedio ponderado, este es asignando según la importancia para el puesto a jugar 

In [8]:
# conexión realizada!
df=pd.read_sql('SELECT * FROM Player_Attributes LIMIT 5',engine)

In [9]:

df

#promedio ponderado con las cualidades
#defensa
#medianamente alto, acceleration y resistencia ,defensa alta, fuerza, ambos taclke, cabezaso, ambos pases mas importante el corto, agression, intercepciom, marca, 2 ambidiestos y o 2izaueirdo y 2 derecho
#centro
# resistencia, agilidad, control pelota, velocidad, pase largo y corto, fuerza, dribiling, intercepciones, stamina, marking, salto, visión
# delantero
# wen atque, velocidad, pique, acelracion , reaccion, salto, control baol, cabezaso, posicionamiento, penalties, shot_power, agilidad, fuerza, pase corto y largo, 
#portero
#reacciomes, diving, marcaje, pie, mano (orden de importancia, solo esta cateroria esta ordewnada)
#defensas=[altura,]

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67,71,right,medium,medium,49,...,54,48,65,69,69,6,11,10,8,8
1,2,218353,505942,2015-11-19 00:00:00,67,71,right,medium,medium,49,...,54,48,65,69,69,6,11,10,8,8
2,3,218353,505942,2015-09-21 00:00:00,62,66,right,medium,medium,49,...,54,48,65,66,69,6,11,10,8,8
3,4,218353,505942,2015-03-20 00:00:00,61,65,right,medium,medium,48,...,53,47,62,63,66,5,10,9,7,7
4,5,218353,505942,2007-02-22 00:00:00,61,65,right,medium,medium,48,...,53,47,62,63,66,5,10,9,7,7


EXTRA:
with player_teams as (
SELECT c.player_api_id,b.team_long_name, b.team_api_id
FROM Match as a
LEFT JOIN Team as b on a.home_team_api_id=b.team_api_id 
LEFT JOIN Player as c on c.player_api_id=a.home_player_1 or c.player_api_id=a.home_player_2 or c.player_api_id=a.home_player_3 or c.player_api_id=a.home_player_4 or c.player_api_id=a.home_player_5 or c.player_api_id=a.home_player_6 or c.player_api_id=a.home_player_7 or c.player_api_id=a.home_player_8 or c.player_api_id=a.home_player_9 or c.player_api_id=a.home_player_10 or c.player_api_id=a.home_player_11
GROUP BY 1,2

union all

SELECT c.player_api_id,b.team_long_name, b.team_api_id
FROM Match as a
LEFT JOIN Team as b on a.home_team_api_id=b.team_api_id 
LEFT JOIN Player as c on c.player_api_id=a.away_player_1 or c.player_api_id=a.away_player_2 or c.player_api_id=a.away_player_3 or c.player_api_id=a.away_player_4 or c.player_api_id=a.away_player_5 or c.player_api_id=a.away_player_6 or c.player_api_id=a.away_player_7 or c.player_api_id=a.away_player_8 or c.player_api_id=a.away_player_9 or c.player_api_id=a.away_player_10 or c.player_api_id=a.away_player_11
GROUP BY 1,2
)